In [45]:
 %matplotlib inline
import pandas as pd
import numpy as np
import itertools
import time
import statsmodels.api as sm
import matplotlib.pyplot as plt
# suppressing warnings
import warnings
warnings.filterwarnings("ignore")

# Subset selection using Python

## Best Subset Selection

In [46]:
df = pd.read_csv('hitters.csv')
df.head()

,Player,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,...,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
0,-Andy Allanson,293,66,1,30,29,14,1,293,66,...,30,29,14,A,E,446,33,20,NaN,A
1,-Alan Ashby,315,81,7,24,38,39,14,3449,835,...,321,414,375,N,W,632,43,10,475.0,N
2,-Alvin Davis,479,130,18,66,72,76,3,1624,457,...,224,266,263,A,W,880,82,14,480.0,A
3,-Andre Dawson,496,141,20,65,78,37,11,5628,1575,...,828,838,354,N,E,200,11,3,500.0,N
4,-Andres Galarraga,321,87,10,39,42,30,2,396,101,...,48,46,33,N,E,805,40,4,91.5,N


In [36]:
print(df['Salary'].isnull().sum())

59


In [37]:
# Drop the player names as they are not a reasonable potential predictor
df = df.drop('Player', axis=1)
# Print the dimensions of the Hitters data (322 rows x 20 columns)(Players' names not included)
print("before dropna():",df.shape)
# Drop any rows the contain missing values. Note that this is not necessarily the recommended practice for a given problem.
df = df.dropna()
# Print the dimensions of the modified Hitters data (263 rows x 20 columns)
print("after dropna():",df.shape)
# One last check: should return 0
print("check the number of missing salary after dropna():",df["Salary"].isnull().sum())

before dropna(): (322, 20)
after dropna(): (263, 20)
check the number of missing salary after dropna(): 0


In [38]:
# assigning dummy variables to non numeric variables
dummies = pd.get_dummies(df[['League', 'Division', 'NewLeague']])
dummies.head()

,League_A,League_N,Division_E,Division_W,NewLeague_A,NewLeague_N
1,0,1,0,1,0,1
2,1,0,0,1,1,0
3,0,1,1,0,0,1
4,0,1,1,0,0,1
5,1,0,0,1,1,0


In [39]:
y = df.Salary
# Drop the column with the dependent variable (Salary), and columns for which we created dummy variables
X_ = df.drop(['Salary', 'League', 'Division', 'NewLeague'], axis=1)
# Define the feature set X.
X = pd.concat([X_, dummies[['League_N', 'Division_W', 'NewLeague_N']]], axis=1)
## Note that alternatively, you can use drop_first=True in .get_dummies to directly get K-1 dummies
## for a categorical variable with K categories.

In [40]:
# Best subset is the one that has the lowest R-sq
def processSubset(feature_set):
    # Fit model on feature_set and calculate RSS
    X1 = sm.add_constant(X[list(feature_set)])
    model = sm.OLS(y,X1)
    regr = model.fit()
    RSS = ((regr.predict(X1) - y) ** 2).sum()
    return {"model":regr, "RSS":RSS}

In [41]:
# Returning the RSS and best subset
def getBest(k):
    tic = time.time()
    results = []
    for combo in itertools.combinations(X.columns, k):
        results.append(processSubset(combo))
    
    # Wrap everything up in a nice dataframe
    models = pd.DataFrame(results)

    # Choose the model with the smallest RSS
    best_model = models.loc[models['RSS'].idxmin()]
    # idxmin() function returns index of first occurrence of minimum.
    
    toc = time.time()
    print("Processed ", models.shape[0], "models on", k, "predictors in",(toc-tic), "seconds.")
    # Return the best model, along with some other useful information about the model
    return best_model

In [42]:
# Implementing Best Subset Selection

# Could take quite awhile to complete...
models = pd.DataFrame(columns=["RSS", "model"])
tic = time.time()
for i in range(0,8):
    models.loc[i] = getBest(i)
    
toc = time.time()
print("Total elapsed time:", (toc-tic), "seconds.")


Processed  1 models on 0 predictors in 0.013671875 seconds.
Processed  19 models on 1 predictors in 0.15624427795410156 seconds.


C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


Processed  171 models on 2 predictors in 1.6842899322509766 seconds.
Processed  969 models on 3 predictors in 9.782030582427979 seconds.
Processed  3876 models on 4 predictors in 39.82716965675354 seconds.
Processed  11628 models on 5 predictors in 127.84610271453857 seconds.
Processed  27132 models on 6 predictors in 284.2820026874542 seconds.
Processed  50388 models on 7 predictors in 512.6112310886383 seconds.
Total elapsed time: 979.0760436058044 seconds.


### Figure out error with professor

## Forward Stepwise Selection

In [48]:
# defining function
def forward(predictors):
    # Pull out predictors we still need to process
    remaining_predictors = [p for p in X.columns if p not in predictors]
    # Initiating timer
    tic = time.time()
    # Initializing lists to store results
    results = []
    
    for p in remaining_predictors:
        results.append(processSubset(predictors+[p]))

    # Wrap everything up in a nice dataframe
    models = pd.DataFrame(results)
    
    # Choosing the model with highest RSS
    best_model = models.loc[models['RSS'].idxmin()]
    # Ending timer
    toc = time.time()
    # Printing performance
    print("Processed ", models.shape[0], "models on", len(predictors)+1,"predictors in", (toc-tic), "seconds.")
    # Returning best model
    return best_model

In [49]:
models2 = pd.DataFrame(columns=["RSS", "model"])

tic = time.time()
predictors = []

for i in range(1,len(X.columns)+1):
    models2.loc[i] = forward(predictors)
    predictors = models2.loc[i]["model"].model.exog_names.copy()
    predictors.remove('const')

toc = time.time()
print("Total elapsed time:", (toc-tic), "seconds.")

Processed  19 models on 1 predictors in 0.14918017387390137 seconds.
Processed  18 models on 2 predictors in 0.1644895076751709 seconds.
Processed  17 models on 3 predictors in 0.1681368350982666 seconds.
Processed  16 models on 4 predictors in 0.15665388107299805 seconds.
Processed  15 models on 5 predictors in 0.18410086631774902 seconds.
Processed  14 models on 6 predictors in 0.17737698554992676 seconds.
Processed  13 models on 7 predictors in 0.17718720436096191 seconds.
Processed  12 models on 8 predictors in 0.14374756813049316 seconds.
Processed  11 models on 9 predictors in 0.1678330898284912 seconds.
Processed  10 models on 10 predictors in 0.14183926582336426 seconds.
Processed  9 models on 11 predictors in 0.11760139465332031 seconds.
Processed  8 models on 12 predictors in 0.12396669387817383 seconds.
Processed  7 models on 13 predictors in 0.10059928894042969 seconds.
Processed  6 models on 14 predictors in 0.10296368598937988 seconds.
Processed  5 models on 15 predictors

## Backward Stepwise Selection

In [50]:
def backward(predictors):
    tic = time.time()
    results = []
    for combo in itertools.combinations(predictors, len(predictors)-1):
        results.append(processSubset(combo))
    # Wrap everything up in a nice dataframe
    models = pd.DataFrame(results)
    # Choose the model with the highest RSS
    best_model = models.loc[models[['RSS']].idxmin()]
    toc = time.time()
    print("Processed ", models.shape[0], "models on", len(predictors)-1,"predictors in", (toc-tic), "seconds.")
    # Return the best model, along with some other useful information about the model
    return best_model

In [51]:
models3 = pd.DataFrame(columns=["RSS", "model"], index = range(1,len(X.columns)))
tic = time.time()
predictors = X.columns
models3.loc[len(predictors)] = getBest(len(predictors))
while(len(predictors) > 1):
    models3.loc[len(predictors)-1] = backward(predictors)
    predictors = models3.loc[len(predictors)-1]["model"].model.exog_names.copy()
    predictors.remove('const')
toc = time.time()
print("Total elapsed time:", (toc-tic), "seconds.")

Processed  1 models on 19 predictors in 0.016264677047729492 seconds.
Processed  19 models on 18 predictors in 0.31957459449768066 seconds.


ValueError: Incompatible indexer with Series

In [52]:
print("Best Subset Selection:\n",models.loc[7, "model"].params)
print("\nForward Stepwise Selection:\n",models2.loc[7, "model"].params)
print("\nBackward Stepwise Selection:\n",models3.loc[7, "model"].params)

Best Subset Selection:
 const          79.450947
Hits            1.283351
Walks           3.227426
CAtBat         -0.375235
CHits           1.495707
CHmRun          1.442054
PutOuts         0.236681
Division_W   -129.986643
dtype: float64

Forward Stepwise Selection:
 const         109.787306
CRBI            0.853762
Hits            7.449877
PutOuts         0.253340
Division_W   -127.122393
AtBat          -1.958885
Walks           4.913140
CWalks         -0.305307
dtype: float64


AttributeError: 'float' object has no attribute 'params'